In [1]:
from google.colab import files
import zipfile, os

In [2]:
uploaded = files.upload()

Saving archive.zip to archive.zip


In [3]:
with zipfile.ZipFile("archive.zip",'r') as zip_ref:
  zip_ref.extractall(".")

In [4]:
data_dir  = "/content/facemask-dataset/dataset"

In [5]:
print(os.listdir(data_dir))

['without_mask', 'with_mask']


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)


In [9]:
train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

test_gen = datagen.flow_from_directory(
    data_dir,
    target_size = (128,128),
    batch_size = 32,
    class_mode = "binary",
    subset = "validation"
)

Found 3067 images belonging to 2 classes.
Found 766 images belonging to 2 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout

In [11]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=["accuracy"])

In [14]:
model.fit(train_gen,validation_data=test_gen,epochs=15)

Epoch 1/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 243ms/step - accuracy: 0.9208 - loss: 0.2176 - val_accuracy: 0.9413 - val_loss: 0.1845
Epoch 2/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 240ms/step - accuracy: 0.9443 - loss: 0.1655 - val_accuracy: 0.9413 - val_loss: 0.1694
Epoch 3/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 237ms/step - accuracy: 0.9467 - loss: 0.1487 - val_accuracy: 0.9517 - val_loss: 0.1431
Epoch 4/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 239ms/step - accuracy: 0.9456 - loss: 0.1553 - val_accuracy: 0.9504 - val_loss: 0.1495
Epoch 5/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 230ms/step - accuracy: 0.9474 - loss: 0.1394 - val_accuracy: 0.9256 - val_loss: 0.2034
Epoch 6/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 236ms/step - accuracy: 0.9485 - loss: 0.1437 - val_accuracy: 0.9426 - val_loss: 0.1530
Epoch 7/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 236ms/step - accuracy: 0.9435 - loss: 0.1655 - val_accuracy: 0.9491 - val_loss: 0.1473
Epoch 8/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 29s 300ms/step - accuracy: 0.9546 - loss: 0.1357 - val_accu

In [16]:
model.save('mask_model.keras')
print("model saved as mask_model.keras")

model saved as mask_model.keras
